In [ ]:
import numpy as np

In [ ]:
class MarketModel:

  #parameters
  #, t, r_hist, R_hist, P_hist, r_cur, s0, s_hist, s_hur, w_hist, w_cur, eta, M, c, mu, d

  #class constructor
  # def __init__(self, T, t, r_hist, R_hist, P_hist, r_cur, s0, s_hist, s_hur, w_hist, w_cur, eta, M, c, mu, d): Most of these are implicitly defined
    # self.d = d
    # self.T = T
    # self.t = t
    # self.r_hist = r_hist
    # self.R_hist = R_hist
    # self.P_hist = P_hist
    # self.r_cur = r_cur
    # self.s0 = s0
    # self.s_hist = s_hist
    # self.s_cur = s_cur
    # self.w_hist = w_hist
    # self.w_cur = w_cur
    # self.eta = eta
    # self.M = M
    # self.c = c
    # self.mu = mu

  #Initialize the market model by specifying the number of time periods, the trade friction parameter, the covariance matrix, the market impact factors, the drift,
  #and the number of assets
  def __init__(self, T=5, eta=0, M=np.array([[1,0],[0,1]]), c=np.array([0,0]), mu=np.array([0,0]), d=2):
    self.d = d
    self.T = T
    self.r_cur = 1
    self.s0 = 100*np.ones((self.d))
    self.w_cur = (1/self.d)*np.ones((self.d))
    self.eta = eta
    self.M = M
    self.c = c
    self.mu = mu
    self.reset()

  def reset(self):
    self.t = 0
    self.s_hist = np.zeros((self.d, self.T+1))
    self.w_hist = np.zeros((self.d, self.T+1))
    self.r_hist = np.zeros((self.T+1))
    self.P_hist = np.zeros((self.T+1))
    self.R_hist = np.zeros((self.T+1))
    self.s_cur = self.s0
    self.s_hist[:,0] = self.s0
    self.w_hist[:,0] = np.ones((self.d))/self.d
    self.R_hist[0] = 1

  def step(self, w):
    if self.t >= self.T:
      print("Trading period terminated (t = T), no more actions may be taken!")

    if np.shape(w)[0] != self.d :
      print("Provided weights are not the correct size: Need more weights")

    elif np.shape(np.shape(w))[0] != 1:
      print("Provided weights are not the correct size: Incorrect Formatting")

    else: # Safety check, don't execute anything if the weights are mis-sized


      # ASSUMES w IS ONE DIMENSIONAL VECTOR (hence the above elif)
      if (w<0).any or np.abs(np.sum(w)-1)<0.00001: # negative weights or not normalized?
        w=forceSimplex(w)

      # Step Forward
      self.t = self.t + 1

      # Update weights
      self.w_cur = w;
      self.w_hist[:, self.t] = self.w_cur

      if self.t == 1:
        u_delta = 0
        self.P_hist[0] = 1

      else:
        self.P_hist[self.t] = ( 1 + self.r_hist[self.t-1])* self.P_hist[self.t - 1]
        u_cur = self.w_cur * self.P_hist[self.t]/self.s_cur
        u_last = self.w_hist[:,self.t-1]*self.P_hist[self.t-1]/self.s_hist[:,self.t-1]
        u_delta = u_cur-u_last

      xi_cur = np.random.normal(loc=0.0, scale=1.0, size=self.d)
      self.s_last = self.s_cur
      self.s_cur = self.genPriceStep(du=u_delta, xi=xi_cur)
      self.s_hist[:,self.t] = self.s_cur
      self.s_cur=self.genPriceStep(du=u_delta, xi=xi_cur)
      r_s_cur = (self.s_cur-self.s_last)/self.s_last;
      self.r_cur = np.dot(self.w_cur,r_s_cur) - self.eta*np.dot(u_delta,u_delta) # Updated 3-18-21: takes change in positions rather than changes in weights
      self.r_hist[self.t] = self.r_cur
      self.R_hist[self.t] = self.getTotalReturn()

  def getTotalReturn(self):
    return (np.prod(self.r_hist+1))

  def genPriceStep (self, du, xi):
    S = np.log(self.s_cur)
    dS = self.mu+self.marketImpact(du)+self.M @ xi
    return np.exp(S+dS)

  def marketImpact(self, du):
    return self.c*np.sign(du)*np.sqrt(np.abs(du))

  def getState(self):
    stateDictionary = {"t":self.t, "s_hist":self.s_hist[:,0:self.t+1], "w_hist":self.w_hist[:,0:self.t+1], "r_hist":self.r_hist[0:self.t+1], "cum_Return":self.R_hist[0:self.t+1]}
    return stateDictionary

  def makeTuple(self):
    stateTuple = {"s":tuple(self.s_hist[:,self.t-1]),"a":tuple(self.w_hist[:,self.t]),"r":self.r_hist[self.t],"sprime":tuple(self.s_hist[:,self.t])}
    return stateTuple


def forceSimplex(weights): # Force all weights to be positive, and normalize all weights
  weights[weights<0]=0
  weights = weights/np.sum(weights)
  return weights

In [ ]:
m = MarketModel(T=5, eta=0, M=np.array([[0.2,-0.1,0],[-0.2,0.2,-0.2],[0,-0.1,0.2]]), c=np.array([0,0,0]), mu=np.array([1,0.00,0.00]), d=3)
m.step(forceSimplex(np.array([1,1,1])))
state = m.getState()
print(state)

{'t': 1, 's_hist': array([[100.        , 249.58946984],
       [100.        ,  70.75028133],
       [100.        , 153.93594858]]), 'w_hist': array([[0.33333333, 0.33333333],
       [0.33333333, 0.33333333],
       [0.33333333, 0.33333333]]), 'r_hist': array([0.        , 2.03322607]), 'cum_Return': array([1.        , 3.03322607])}


In [ ]:
print(m.makeTuple())

{'s': (100.0, 100.0, 100.0), 'a': (0.3333333333333333, 0.3333333333333333, 0.3333333333333333), 'r': 2.033226067586663, 'sprime': (249.5894698376738, 70.75028133079665, 153.93594858044688)}


In [ ]:
state=m.getState()
w=state["s_hist"][:,state["t"]]-state["s_hist"][:,state["t"]-1]
print(w)
w=forceSimplex(w)
print(w)

[149.58946984 -29.24971867  53.93594858]
[0.73499158 0.         0.26500842]


In [ ]:
print(m.getState())

{'t': 1, 's_hist': array([[100.        , 249.58946984],
       [100.        ,  70.75028133],
       [100.        , 153.93594858]]), 'w_hist': array([[0.33333333, 0.33333333],
       [0.33333333, 0.33333333],
       [0.33333333, 0.33333333]]), 'r_hist': array([0.        , 2.03322607]), 'cum_Return': array([1.        , 3.03322607])}


https://github.com/SIAM-FM21-PC/MathWorks/blob/main/README.md is the GitHub repository of the SIAM FE

In [ ]:
x= np.array([1,2])

In [ ]:
print(tuple(x))

(1, 2)
